<a href="https://colab.research.google.com/github/alik604/Notebooks/blob/master/Recommendation%20systems/Boston_housing_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier


from sklearn import metrics

boston = load_boston()
import matplotlib.pyplot as plt

%matplotlib inline


## Loading data and pre-processing 


In [2]:
df_boston = pd.DataFrame(boston.data,columns=boston.feature_names)
df_boston['target'] = pd.Series(boston.target)
df = df_boston
df.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


| Feature | Description                                                           |
|---------|-----------------------------------------------------------------------|
| crim    | per capita crime rate by town.                                        |
| zn      | proportion of residential land zoned for lots over 25,000     |
|         |                                                                       |
| indus   | proportion of non-retail business acres per town.                     |
| chas    | Charles River dummy variable (= 1 if tract bounds river; 0otherwise|
|         |                                                                       |
| nox     | nitrogen oxides concentration (parts per 10 million).                 |
| lstat   | median value of owner-occupied homes in $$ 1000s.                       |
| rm      | average number of rooms per dwelling.                                 |
| age     | proportion of owner-occupied units built prior to 1940.               |
| dis     | weighted mean of distances to five Boston employment centres.         |
| rad     | index of accessibility to radial highways.                            |
| tax     | full-value property-tax rate per \$10,000.                             |
| ptratio | pupil-teacher ratio by town.                                          |
| black   | 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.       |
| lstat   | lower   status of the population (percent).                           |
| medv    | median   value of owner-occupied homes in $1000s.                     |

###  surely no one cares about nox,chas 

In [39]:
df =df_boston.drop(['NOX','CHAS'],axis=1)
df_byCRIM = df.sort_values('CRIM')

own = np.zeros(len(df_byCRIM))
own[:10] = 1 
df_byCRIM['own'] = own

df = df_byCRIM.sort_index()
df.head(15)

,CRIM,ZN,INDUS,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target,own
0,0.00632,18.0,2.31,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,1.0
1,0.02731,0.0,7.07,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,0.0
2,0.02729,0.0,7.07,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,0.0
3,0.03237,0.0,2.18,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,0.0
4,0.06905,0.0,2.18,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,0.0
5,0.02985,0.0,2.18,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,0.0
6,0.08829,12.5,7.87,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,0.0
7,0.14455,12.5,7.87,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,0.0
8,0.21124,12.5,7.87,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,0.0
9,0.17004,12.5,7.87,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,0.0


In [0]:
if len(df_byCRIM) > 10:
   df_client = df_byCRIM[:10]
   df_listings = df_byCRIM[10:]

In [67]:
df_new = pd.concat([df_client, df] ,axis=0)
df_new.head(14)

,CRIM,ZN,INDUS,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target,own
0,0.00632,18.0,2.31,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,1.0
284,0.00906,90.0,2.97,7.088,20.8,7.3073,1.0,285.0,15.3,394.72,7.85,32.2,1.0
285,0.01096,55.0,2.25,6.453,31.9,7.3073,1.0,300.0,15.3,394.72,8.23,22.0,1.0
341,0.01301,35.0,1.52,7.241,49.3,7.0379,1.0,284.0,15.5,394.74,5.49,32.7,1.0
55,0.01311,90.0,1.22,7.249,21.9,8.6966,5.0,226.0,17.9,395.93,4.81,35.4,1.0
54,0.01360,75.0,4.00,5.888,47.6,7.3197,3.0,469.0,21.1,396.90,14.80,18.9,1.0
195,0.01381,80.0,0.46,7.875,32.0,5.6484,4.0,255.0,14.4,394.23,2.97,50.0,1.0
57,0.01432,100.0,1.32,6.816,40.5,8.3248,5.0,256.0,15.1,392.90,3.95,31.6,1.0
194,0.01439,60.0,2.93,6.604,18.8,6.2196,1.0,265.0,15.6,376.70,4.38,29.1,1.0
348,0.01501,80.0,2.01,6.635,29.7,8.3440,4.0,280.0,17.0,390.94,5.99,24.5,1.0


In [65]:
X = df_new.drop(['own'],axis=1) # Features
y = df_new['own'] # Target variable

def non_shuffling_train_test_split(X, y, test_size):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y, test_size=0.85)

X_train.head(3)

,CRIM,ZN,INDUS,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
284,0.00906,90.0,2.97,7.088,20.8,7.3073,1.0,285.0,15.3,394.72,7.85,32.2
285,0.01096,55.0,2.25,6.453,31.9,7.3073,1.0,300.0,15.3,394.72,8.23,22.0


## Testing models


In [60]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.9931506849315068


In [0]:
y_pred

In [0]:
np.array(y_test)

In [68]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=8)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.9931506849315068


In [0]:
from sklearn.svm import LinearSVC

In [69]:
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.3835616438356164


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
# y_pred

In [0]:
# np.array(y_test)

In [70]:
from sklearn.svm import OneClassSVM
clf = OneClassSVM()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.0091324200913242


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [79]:
from sklearn.neighbors import KNeighborsClassifier

for i in range(1,15):
  knn = KNeighborsClassifier(n_neighbors=i)
  knn.fit(X_train, y_train)
  y_pred = knn.predict(X_test)
  print('I: {0}, Accuracy: {1}'.format(i, metrics.accuracy_score(y_test, y_pred)))


I: 1, Accuracy: 0.6484018264840182
I: 2, Accuracy: 0.6506849315068494
I: 3, Accuracy: 0.632420091324201
I: 4, Accuracy: 0.9726027397260274
I: 5, Accuracy: 0.9703196347031964
I: 6, Accuracy: 0.9703196347031964
I: 7, Accuracy: 0.95662100456621
I: 8, Accuracy: 0.958904109589041
I: 9, Accuracy: 0.954337899543379
I: 10, Accuracy: 0.9726027397260274
I: 11, Accuracy: 0.9703196347031964
I: 12, Accuracy: 0.9726027397260274
I: 13, Accuracy: 0.9703196347031964
I: 14, Accuracy: 0.9748858447488584


In [83]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('I: {0}, Accuracy: {1}'.format(i, metrics.accuracy_score(y_test, y_pred)))


I: 14, Accuracy: 0.9703196347031964


In [0]:
y_pred

In [0]:
np.array(y_test)